# Exercices Extra : Preprocessing 

Dans ce Notebook, des exemples de chose pratiques qu'on est souvent amené à faire 
avant de pouvoir analyser de manière appropriée nous données 

## Nettoyage : Ajuster les noms des colonnes

Parfois les colonnes dans nos données ont de noms qui soient représentatifs (ou faciles à comprendre). Il arrive même que certains noms de colonnes comportent des "*erreurs*" (des caractères spéciaux, des espaces, etc.). 

On peut modifier les noms des colonnes d'un **DataFrame** facilement. 


Afin d'illustrer le changement de colonne, prenons le fichier "***VentesAgenceU.csv***". On va :
- lire le fichier ***VentesAgenceU.csv***
- faire un ***info*** pour vérifier les colonnes que nous avons

In [ ]:
import pandas as pnd

dfFactures = pnd.read_csv ('datasets/VentesAgenceU.csv',
                           delimiter=';', header=[0], index_col=[0] ) 

dfFactures.info()



On va supprimer la colonne n° 5 ("*Unnamed: 6*") et les lignes avec des NaN, afin de ne garder que les données "utiles". 

In [ ]:
dfFactures.drop(columns={dfFactures.columns[5]}, inplace=True)
dfFactures.dropna(axis='index', inplace=True)
dfFactures.info()

On observe que nom de la ***" MONTANT "*** contient des espaces, ce qui n'est pas très pratique. 

Afin de bien voir les possibles espaces dans le nom des colonnes, on va utiliser une petite boucle, qui va afficher les noms de chaque colonne précédée par un ##. 

In [ ]:
for col in dfFactures.columns :
    print('##{}##'.format(col))

On voit que le nom de la colonne "*MONTANT*" possède des espaces avant et après, ce qui peut être très gênant au moment qu'on voudra manipuler cette colonne. 

On va donc modifier son nom avec l'opération ***rename***.

In [ ]:
# 1ere possibilité, en se servant de l'attribut columns 
# sans le inplace pour le pas modifier le DataFrame, et donc pouvoir l'application des deux options :-)
dfFactures.rename (columns= { dfFactures.columns[4] : 'montant' } )

In [ ]:
# maintenant avec le inplace pour modifier le DataFrame
dfFactures.rename(columns={' MONTANT ':'MONTANT'}, inplace=True)

for col in dfFactures.columns :
    print('##{}##'.format(col))
    
dfFactures.info()

## Nettoyage : Eliminer les caractères indésirables

Parfois il faut supprimer des données des caractères indésirables (un "$" ou un "€" d'un prix, par exemple), ou les remplacer par d'autres (parfois il faut remplacer une "," d'un prix en format FR par un "." d'un format US compris comme un numéro par Python). 

Cette étape doit venir avant qu'on puisse convertir les colonnes contenant ces données dans les bons formats (numeric, date). 

On peut remplacer un caractère par un autre avec ***str.replace*** ou en utilisant l'opération ***apply***. Si on veut tout simplement supprimer le caractère, il suffit de le remplacer par ***''*** (une chaine "*vide*"). 

On va illustrer cela avec la colonne "***MONTANT***" du DataFrame "***dfFractures***". 
En faisant un ***sample***, on peut observer l'usage de la "," comme séparateur pour les cases décimales, et plusieurs valeurs avec un "_". Ceci explique pourquoi cette colonne apparaît comme ***object*** dans la sortie de l'opération ***info***. 


In [ ]:
dfFactures.sample(5)

On va donc :
- remplacer le '_' par '' (chaîne vide)
- remplacer le ',' par un '.' 

In [ ]:
dfFactures['MONTANT'] = dfFactures['MONTANT'].str.replace('_','')
dfFactures['MONTANT'].sample(5)

Même chose avec l'opération ***apply***. 

In [ ]:
dfFactures['MONTANT'] = dfFactures['MONTANT'].apply (lambda x: str(x).replace(',' , '.') )
dfFactures['MONTANT'].sample(5)

Dès que les données ont été nettoyées, on peut le convertir en type "*numérique*" avec ***to_numeric***.  

In [ ]:
dfFactures['MONTANT'] = pnd.to_numeric(dfFactures['MONTANT'],errors='coerce')
dfFactures.info()

## Dates : obtenir plus d'informations sur les dates. 

A partir du moment qu'on convertit une colonne en date (***to_datetime***), on peut obtenir plusieurs informations à partir de ses valeurs. 


In [ ]:
dfFactures.info()

La colonne "*DATE FACTURE*" contient des dates. Il faut donc la convertir avec ***to_datetime***.  

In [ ]:
dfFactures['DATE FACTURE'] = pnd.to_datetime(dfFactures['DATE FACTURE'])
dfFactures['DATE FACTURE'].sample(5)

A partir de ce moment, on va pouvoir en extraire des nombreuses informations utiles :
- jour 
- mois 
- année
- jour de la semaine
- jour de la semaine en texte
- mois en texte

In [ ]:
dfFactures['JOUR FACTURE'] = dfFactures['DATE FACTURE'].dt.day
dfFactures['MOIS FACTURE'] = dfFactures['DATE FACTURE'].dt.month
dfFactures['ANNEE FACTURE'] = dfFactures['DATE FACTURE'].dt.year
dfFactures['JOUR SEMAINE'] = dfFactures['DATE FACTURE'].dt.weekday
dfFactures['NOM JOUR'] = dfFactures['DATE FACTURE'].dt.day_name()
dfFactures['NOM MOIS'] = dfFactures['DATE FACTURE'].dt.month_name()

dfFactures['SEMAINE ANNEE'] = dfFactures['DATE FACTURE'].dt.isocalendar().week

dfFactures.sample(5)

Si on essaie de récupérer l'heure dans les données de la colonne "*DATE_FACTURE*", on ne pourra pas, car cette information est absente des données de base. 

In [ ]:
dfFactures['DATE FACTURE'].dt.time